In [ ]:
import os
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.3,org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.3 pyspark-shell'

from pyspark import SparkConf
from pyspark import  SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SQLContext
import pyspark

In [ ]:
spark_conf = SparkConf() \
            .setMaster("spark://spark-master:7077") \
            .setAppName("SparkKafkaStreamingConsole") \
            .setAll([('spark.executor.memory', '4g'),
                     ('spark.executor.cores', '4'), 
                     ('spark.cores.max', '4'), 
                     ('spark.driver.memory','4g'),
                     ('spark.driver.host','192.168.96.3')
            ])

spark = SparkSession.builder \
                    .config(conf=spark_conf) \
                    .enableHiveSupport() \
                    .getOrCreate()

In [ ]:
input_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:9092") \
        .option("subscribe", "topic1") \
        .option("startingOffsets", "latest") \
        .load() \
        .selectExpr("CAST(value as STRING)", "CAST(timestamp as STRING)","CAST(topic as STRING)")

In [ ]:
output = input_df.writeStream \
            .format("console") \
            .option("truncate","false") \
            .start() \
            .awaitTermination()

In [ ]:
def SaveToHBase(rdd):
#     print("=====Pull from Stream=====")
    if not rdd.isEmpty():
        host = 'localhost:2182'
        table = 'logs'  
        keyConv = "org.apache.spark.examples.pythonconverters.StringToImmutableBytesWritableConverter"  
        valueConv = "org.apache.spark.examples.pythonconverters.StringListToPutConverter"  
        conf = {"hbase.zookeeper.quorum": host,
                "hbase.zookeeper.property.clientPort": "2182",
            "hbase.mapred.outputtable": table,
            "mapreduce.outputformat.class": "org.apache.hadoop.hbase.mapreduce.TableOutputFormat",  
            "mapreduce.job.output.key.class": "org.apache.hadoop.hbase.io.ImmutableBytesWritable",  
            "mapreduce.job.output.value.class": "org.apache.hadoop.io.Writable"}
        rdd.saveAsNewAPIHadoopDataset(conf=conf,keyConverter=keyConv,valueConverter=valueConv)